In [8]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")

from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbike(RootModel):
    root:list[Site]


data = Youbike.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df

下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:29:21,復興南路二段235號前,True,28,8,25.02605,121.54360,20
1,復興南路二段273號前,大安區,2024-06-17 19:29:21,復興南路二段273號西側,True,21,12,25.02565,121.54357,9
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.02429,121.54124,13
3,和平公園東側,大安區,2024-06-17 19:29:21,和平東路二段118巷33號,True,11,11,25.02351,121.54282,0
4,辛亥復興路口西北側,大安區,2024-06-17 19:28:20,復興南路二段368號,True,16,5,25.02153,121.54299,11
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:29:21,臺大霖澤館南側,True,20,2,25.01995,121.54347,18
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,True,24,1,25.01791,121.54242,23
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:28:20,臺大體育館東側,True,40,7,25.02112,121.53591,33


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   行政區域    1423 non-null   object 
 2   時間      1423 non-null   object 
 3   位置      1423 non-null   object 
 4   狀態      1423 non-null   bool   
 5   總數量     1423 non-null   int64  
 6   可借      1423 non-null   int64  
 7   緯度      1423 non-null   float64
 8   經度      1423 non-null   float64
 9   可還      1423 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [6]:
import numpy as np

less3_df = df.query('可借<=3')
less3_df.shape

(423, 10)

In [11]:
df2 = df.query('可借<=3 or 可還<=10')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
1,復興南路二段273號前,大安區,2024-06-17 19:29:21,復興南路二段273號西側,True,21,12,25.02565,121.54357,9
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.02429,121.54124,13
3,和平公園東側,大安區,2024-06-17 19:29:21,和平東路二段118巷33號,True,11,11,25.02351,121.54282,0
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,True,11,10,25.02429,121.54328,1
6,復興南路二段340巷口,大安區,2024-06-17 19:28:20,復興南路二段342號,True,11,7,25.02253,121.54326,4
...,...,...,...,...,...,...,...,...,...,...
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:25:22,臺大圖書資訊館北側,True,25,1,25.01804,121.53928,24
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:29:21,臺大霖澤館南側,True,20,2,25.01995,121.54347,18
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,True,24,1,25.01791,121.54242,23


In [13]:
df3 = df.query('行政區域=="大安區" and 可借==0')
df3

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.53310,5
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:29:21,基隆路三段155巷57號西側,True,20,0,25.014260,121.54382,18
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.015760,121.54291,18
35,臺大社科院圖書館前,大安區,2024-06-17 19:28:20,辛亥路與復興南路口西南側,True,27,0,25.021010,121.54284,27
41,和平實小東北側,大安區,2024-06-17 19:29:15,敦南街73號,True,13,0,25.021641,121.54704,13
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.022150,121.54839,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.027210,121.54988,17
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,False,20,0,25.042010,121.54612,20
102,東豐公園,大安區,2024-06-17 19:21:15,東豐街24號(旁公園路側),True,19,0,25.035690,121.54510,19
104,捷運忠孝復興站(5號出口),大安區,2024-01-14 04:19:10,復興南路一段135巷3號(對面),False,41,0,25.042620,121.54443,41


In [14]:
n = 0
df4 = df.query('行政區域=="大安區" and 可借==@n')
df4

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.53310,5
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:29:21,基隆路三段155巷57號西側,True,20,0,25.014260,121.54382,18
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.015760,121.54291,18
35,臺大社科院圖書館前,大安區,2024-06-17 19:28:20,辛亥路與復興南路口西南側,True,27,0,25.021010,121.54284,27
41,和平實小東北側,大安區,2024-06-17 19:29:15,敦南街73號,True,13,0,25.021641,121.54704,13
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.022150,121.54839,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.027210,121.54988,17
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,False,20,0,25.042010,121.54612,20
102,東豐公園,大安區,2024-06-17 19:21:15,東豐街24號(旁公園路側),True,19,0,25.035690,121.54510,19
104,捷運忠孝復興站(5號出口),大安區,2024-01-14 04:19:10,復興南路一段135巷3號(對面),False,41,0,25.042620,121.54443,41
